In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Pro2/KmeanData.csv')

In [4]:
data

,headline,title,text
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess..."
1,"\nGet a bachelor’s degree.,\nEnroll in a studi...",How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
2,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ..."
3,\nKeep all of your past work organized and acc...,How to Be an Organized Artist3,"When you finish a project, whether it sells o..."
4,"\nCreate a compelling reel or portfolio.,\nLan...",How to Be a Visual Effects Artist2,This should be a short video showcasing the b...
...,...,...,...
78963,"\nUse a childhood nickname.,\nUse your middle ...",How to Pick a Stage Name2,You may have been called something other than...
78964,\nConsider changing the spelling of your name....,How to Pick a Stage Name3,"If you have a name that you like, you might f..."
78965,"\nTry out your name.,\nDon’t legally change yo...",How to Pick a Stage Name4,Your name might sound great to you when you s...
78966,\nUnderstand the process of intaglio printing....,How to Identify Prints2,"Intaglio is Italian for ""incis­ing,"" and corr..."


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
vocab = word_vectors.vocab

In [ ]:
from nltk.tokenize import word_tokenize
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min


n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters)

In [ ]:
!pip install rouge-score
!pip install rouge/requirements.txt
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL','rouge2'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
scores

In [ ]:
def clearData(content):
    content = content.lower() #Biến đổi hết thành chữ thường
    content = content.replace('\n', ' ') #Đổi các ký tự xuống dòng thành chấm câu
    content = content.replace(',', '') 
    content = content.replace(':', '')
    content = content.strip()
    return content

In [ ]:
sum_rouge1 = sum_rougeL=sum_rouge2 = 0
can_path = '/content/drive/MyDrive/Pro2/Kmean/candidate'
gold_path= '/content/drive/MyDrive/Pro2/Kmean/gold'
with open(can_path, 'w') as save_pred:
    with open(gold_path, 'w') as save_gold:
        for x in range(25000):
            content = clearData(data.iloc[x]['text'])

            sentences = nltk.sent_tokenize(content)
            if(len(sentences)<5):
                continue
            X = []
            for sentence in sentences:
                sentence_tokenized = word_tokenize(sentence)
                # words = sentence_tokenized.split(" ")
                sentence_vec = np.zeros((100))
                for word in sentence_tokenized:
                    if word in vocab:
                        sentence_vec += word_vectors[word]
                X.append(sentence_vec)
            kmeans = kmeans.fit(X)
            avg = []
            for j in range(n_clusters):
                idx = np.where(kmeans.labels_ == j)[0]
                avg.append(np.mean(idx))
            closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
            ordering = sorted(range(n_clusters), key=lambda k: avg[k])
            summary = ' '.join([sentences[closest[idx]] for idx in ordering])
            summary = summary.replace('.',"")
                
            gold = clearData(data.iloc[x]['headline']).replace(".",'')
            gold = gold.replace('"',"")

            # save_pred.write(summary.strip() +'\n')
            # save_gold.write(gold.strip() +'\n')
            scores = scorer.score(summary,gold)
            if x % 1000 ==0:
                print( scores['rouge1'][2],scores['rougeL'][2],scores['rouge2'][2])
            sum_rouge1  += scores['rouge1'][2]
            sum_rougeL += scores['rougeL'][2]
            sum_rouge2 += scores['rouge2'][2]

0.3067484662576687 0.147239263803681 0.02484472049689441
0.28099173553719003 0.17355371900826447 0.06666666666666667
0.18965517241379312 0.13793103448275862 0.0
0.3305785123966943 0.21487603305785127 0.11764705882352941
0.3203883495145632 0.17475728155339806 0.11764705882352941
0.396551724137931 0.2586206896551724 0.05263157894736842
0.22818791946308728 0.1476510067114094 0.0816326530612245
0.2417582417582418 0.21978021978021975 0.0898876404494382
0.18181818181818185 0.12987012987012989 0.06578947368421052
0.3896103896103896 0.2077922077922078 0.11842105263157894
0.2028985507246377 0.11594202898550722 0.0
0.13095238095238096 0.08333333333333331 0.012048192771084336
0.36363636363636365 0.2644628099173553 0.18487394957983194
0.30303030303030304 0.16666666666666669 0.061538461538461535
0.24427480916030533 0.13740458015267176 0.031007751937984492
0.2521008403361344 0.11764705882352941 0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.2933333333333333 0.17333333333333334 0.04054054054054054
0.24390243902439024 0.19512195121951223 0.07407407407407407
0.24309392265193366 0.12154696132596683 0.0446927374301676
0.14778325123152708 0.12807881773399016 0.05970149253731344
0.25316455696202533 0.16455696202531644 0.07692307692307693
0.2619047619047619 0.15476190476190477 0.024096385542168672
0.176 0.112 0.0
0.3188405797101449 0.14492753623188406 0.04411764705882353
0.23188405797101447 0.14492753623188406 0.044117647058823525


In [ ]:
print(sum_rouge1/25000*100)
print(sum_rougeL/25000*100)
print(sum_rouge2 /25000*100)

26.344593697585307
15.560341619539642
6.227496136942368
